In [3]:
import csv
import math
import pandas as pd
from scipy.stats import poisson
from scipy.stats import laplace
from scipy.stats import gennorm
from diffprivlib.mechanisms import Laplace
import matplotlib.pyplot as plt

In [4]:
df = pd.read_csv('./Sample Data/R01E01D01.csv')
print(df.columns)

Index(['ID', 'Form', '1st reduction', '2nd reduction', 'trial', 'duration',
       '3D deviation', 'XY deviation', 'XY directed', 'Z deviation',
       'Z directed', 'Z std deviation '],
      dtype='object')


In [8]:
if 'ID' in df.columns:
    print({'values': df['ID'].unique().tolist()})

{'values': [1, 2, 3, 4, 5, 6, 7, 9, 10, 11, 13, 14, 15, 16, 17, 18]}


In [39]:
def get_column(column):
    if column not in df.columns:
        return "Not a valid column"
    return {column: df[column].unique()}

def rewrite_column(column, n_title, values):
    global df
    old_values = df[column].unique()
    mapping = {}
    for i in range(len(values)):
        mapping[old_values[i]] = values[i]
    
    df[column] = df[column].map(mapping)
    df = df.rename(columns={column:n_title})
    
rewrite_column('', 'Pill', ['Green', 'Blue'])
print(df.head())

   ID   Form 1st reduction 2nd reduction  trial     duration  3D deviation  \
0   1  Green       virtual          line      1  6291.260010     10.877423   
1   1  Green       virtual          line      2  4419.739990     13.996290   
2   1  Green       virtual          line      3  7070.550049     34.647458   
3   1  Green       virtual          line      4  5209.300049     11.654907   
4   1  Green       virtual          line      5  4249.080078     12.191439   

   XY deviation  XY directed  Z deviation  Z directed  Z std deviation   
0      5.394565     1.004740     8.196941    8.142806          7.473217  
1      9.036742    -3.674751     9.658320    9.306156          7.571380  
2     27.119768    18.843695    16.397204   15.888222         11.080598  
3      8.650802    -7.229488     7.001513    2.840199          3.956877  
4      9.049952    -6.689876     7.569806    2.057051          4.729872  


In [29]:
def value_obf(column):
    laplace = Laplace()
    if df[column].dtype == 'int64':
        laplace.set_epsilon(1)
        laplace.set_sensitivity(df[column].mean()*0.1)
        df[column] = df[column].apply(laplace.randomise).round(0).astype(int)
    elif df[column].dtype == 'object':
        vals = df[column].unique()
        mapping = dict([(y,x) for x,y in enumerate(sorted(set(vals)))]) 
        rev_map = {k: v for k, v in enumerate(mapping)}
        print(rev_map)
        df[column] = df[column].map(mapping)
        laplace.set_epsilon(1)
        laplace.set_sensitivity(df[column].mean()*0.5)
        df[column] = df[column].apply(laplace.randomise).round(0).astype(int)
        print(rev_map)
        df[column] = df[column].map(rev_map)
    elif df[column].dtype == 'float64':
        laplace.set_epsilon(1)
        laplace.set_sensitivity(df[column].mean()*0.1)
        df[column] = df[column].apply(laplace.randomise)
    else:
        print(column + ' Datatype not obfuscatable')
